In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import time
import json
import seaborn
from scipy.stats import ttest_ind

#yelp API data

ykey = '<INSERT KEY HERE>'
yelp_id = '<INSERT ID HERE'
yelp_url = "https://api.yelp.com/v3/businesses/search"
headers = {'Authorization': 'Bearer {}'.format(ykey)}

#data is split into two dataframes because Yelp API has a limit of 5000 
fastfood_data_1 = pd.read_csv('Resources/cleaned_fastfood_data_p1.csv')
fastfood_data_2 = pd.read_csv('Resources/cleaned_fastfood_data_p2.csv')
fastfood_data_1.head()


,name,categories,address,city,postalCode,state,country,latitude,longitude,count
0,Carl's Jr.,"Fast Food Restaurants,Hamburgers and Hot Dogs,...",2555 11th Avenue,Greeley,80631,CO,US,40.39629,-104.69699,205
1,Del Taco,"Restaurant,Mexican Restaurants,Fast Food Resta...",2513 Highway 6 And 50,Grand Junction,81505,CO,US,39.08135,-108.58689,30
2,Chipotle Mexican Grill,"Fast Food Restaurants,Mexican Restaurants,Rest...",3455 N Salida Court,Aurora,80011,CO,US,39.76369,-104.77671,142
3,Taco Bell,"Fast Food Restaurants,Mexican Restaurants,Rest...",5225 E Colfax Avenue,Denver,80220,CO,US,39.74044,-104.92636,529
4,McDonald's,"Fast Food Restaurants,Hamburgers and Hot Dogs,...",1350 W Colfax Avenue,Denver,80204,CO,US,39.73967,-105.00467,784


In [2]:
#renaming state abbreviation to the full state name so it will work with Census API
renamed_fastfood_data_1 = fastfood_data_1.replace({
    'AL' : 'Alabama',
    'AK' : 'Alaska',
    'AZ' : 'Arizona',
    'AR' : 'Arkansas',
    'CA' : 'California',
    'CO' : 'Colorado',
    'CT' : 'Connecticut',
    'DE' : 'Delaware',
    'FL' : 'Florida',
    'GA' : 'Georgia',
    'HI' : 'Hawaii',
    'ID' : 'Idaho',
    'IL' : 'Illinois',
    'IN' : 'Indiana',
    'IA' : 'Iowa',
    'KS' : 'Kansas',
    'KY' : 'Kentucky',
    'LA' : 'Louisiana',
    'ME' : 'Maine',
    'MD' : 'Maryland',
    'MA' : 'Massachusetts',
    'MI' : 'Michigan',
    'MN' : 'Minnesota',
    'MS' : 'Mississippi',
    'MO' : 'Missouri',
    'MT' : 'Montana',
    'NE' : 'Nebraska',
    'NV' : 'Nevada',
    'NH' : 'New Hampshire',
    'NJ' : 'New Jersey',
    'NM' : 'New Mexico',
    'NY' : 'New York',
    'NC' : 'North Carolina',
    'ND' : 'North Dakota',
    'OH' : 'Ohio',
    'OK' : 'Oklahoma',
    'OR' : 'Oregon',
    'PA' : 'Pennsylvania',
    'RI' : 'Rhode Island',
    'SC' : 'South Carolina',
    'SD' : 'South Dakota',
    'TN' : 'Tennessee',
    'TX' : 'Texas',
    'UT' : 'Utah',
    'VT' : 'Vermont',
    'VA' : 'Virginia',
    'WA' : 'Washington',
    'WV' : 'West Virginia',
    'WI' : 'Wisconsin',
    'WY' : 'Wyoming'})
renamed_fastfood_data_2 = fastfood_data_2.replace({
    'AL' : 'Alabama',
    'AK' : 'Alaska',
    'AZ' : 'Arizona',
    'AR' : 'Arkansas',
    'CA' : 'California',
    'CO' : 'Colorado',
    'CT' : 'Connecticut',
    'DE' : 'Delaware',
    'FL' : 'Florida',
    'GA' : 'Georgia',
    'HI' : 'Hawaii',
    'ID' : 'Idaho',
    'IL' : 'Illinois',
    'IN' : 'Indiana',
    'IA' : 'Iowa',
    'KS' : 'Kansas',
    'KY' : 'Kentucky',
    'LA' : 'Louisiana',
    'ME' : 'Maine',
    'MD' : 'Maryland',
    'MA' : 'Massachusetts',
    'MI' : 'Michigan',
    'MN' : 'Minnesota',
    'MS' : 'Mississippi',
    'MO' : 'Missouri',
    'MT' : 'Montana',
    'NE' : 'Nebraska',
    'NV' : 'Nevada',
    'NH' : 'New Hampshire',
    'NJ' : 'New Jersey',
    'NM' : 'New Mexico',
    'NY' : 'New York',
    'NC' : 'North Carolina',
    'ND' : 'North Dakota',
    'OH' : 'Ohio',
    'OK' : 'Oklahoma',
    'OR' : 'Oregon',
    'PA' : 'Pennsylvania',
    'RI' : 'Rhode Island',
    'SC' : 'South Carolina',
    'SD' : 'South Dakota',
    'TN' : 'Tennessee',
    'TX' : 'Texas',
    'UT' : 'Utah',
    'VT' : 'Vermont',
    'VA' : 'Virginia',
    'WA' : 'Washington',
    'WV' : 'West Virginia',
    'WI' : 'Wisconsin',
    'WY' : 'Wyoming'})
renamed_fastfood_data_2.head()

,name,categories,address,city,postalCode,state,country,latitude,longitude,count
0,Panera Bread,"Breakfast,Cafe,Bakery,Caf,Take Out Restaurants...",3942 Townsfair Way,Columbus,43219,Ohio,US,40.049258,-82.916792,140
1,Arby's,"Delis And Delicatessens,Restaurant,Fast Food R...",603 S George Nigh Expy,McAlester,74501,Oklahoma,US,34.916276,-95.746002,252
2,Pizza Hut,"Carry-out food,Caterers,Restaurant delivery se...",1831 Beall Ave,Wooster,44691,Ohio,US,40.821870,-81.933747,474
3,Pizza Hut,"Pizza,Carry-out food,Restaurant,Caterers,Resta...",14923 Pacific Ave S,Tacoma,98444,Washington,US,47.121221,-122.434370,474
4,Pizza Hut,"Carry-out food,Caterers,Restaurant delivery se...",786 S Church St,Forest City,28043,North Carolina,US,35.319737,-81.880982,474


In [3]:
#test block using first restaurant from first dataframe
params = {
    'term' : 'Carls Jr',
    'latitude' : 40.396290,
    'longitude' : -104.696990,
}
yelp_response = requests.get(yelp_url, headers = headers, params = params)
yelp_json = yelp_response.json()
print(json.dumps(yelp_json, indent = 4, sort_keys = True))

{
    "businesses": [
        {
            "alias": "carls-jr-greeley",
            "categories": [
                {
                    "alias": "hotdogs",
                    "title": "Fast Food"
                },
                {
                    "alias": "burgers",
                    "title": "Burgers"
                }
            ],
            "coordinates": {
                "latitude": 40.395724682959894,
                "longitude": -104.6973388
            },
            "display_phone": "(970) 356-9611",
            "distance": 69.45431896107299,
            "id": "_xoPpvH5YEMKy05IaLJ-tg",
            "image_url": "https://s3-media1.fl.yelpcdn.com/bphoto/4UNvWSqPG-C3YPxaQR7F8w/o.jpg",
            "is_closed": false,
            "location": {
                "address1": "2555 11th Ave",
                "address2": null,
                "address3": null,
                "city": "Greeley",
                "country": "US",
                "display_address": [
          

In [3]:
#adding columns to dataframe for Rating, Review Count, and if the restaurant is closed
renamed_fastfood_data_1['Rating'] = ""
renamed_fastfood_data_1['Review Count'] = ""
renamed_fastfood_data_1['Closed'] = False
renamed_fastfood_data_2['Rating'] = ""
renamed_fastfood_data_2['Review Count'] = ""
renamed_fastfood_data_2['Closed'] = False



In [5]:
#adding data from Yelp API into the first dataframe
params = {}
counter = 0
for index, row in renamed_fastfood_data_1.iterrows():
    restaurant_name = row['name']
    restaurant_lat = row['latitude']
    restaurant_lng = row['longitude']
    params['term'] = restaurant_name
    params['latitude'] = restaurant_lat
    params['longitude'] = restaurant_lng
    try:
        yelp_response = requests.get(yelp_url, headers = headers, params = params, timeout = 5)
        yelp_json = yelp_response.json()
        renamed_fastfood_data_1.loc[index, 'Rating'] = yelp_json['businesses'][0]['rating']
        renamed_fastfood_data_1.loc[index, 'Review Count'] = yelp_json['businesses'][0]['review_count']
        renamed_fastfood_data_1.loc[index, 'Closed'] = yelp_json['businesses'][0]['is_closed']
        counter = counter + 1
    except (KeyError, IndexError):
        renamed_fastfood_data_1.loc[index, 'Closed'] = True
        continue
renamed_fastfood_data_1.to_csv('Resources/fastfood_yelp_1.csv')
renamed_fastfood_data_1

,name,categories,address,city,postalCode,state,country,latitude,longitude,count,Rating,Review Count,Closed
0,Carl's Jr.,"Fast Food Restaurants,Hamburgers and Hot Dogs,...",2555 11th Avenue,Greeley,80631,Colorado,US,40.396290,-104.696990,205,2.5,6,False
1,Del Taco,"Restaurant,Mexican Restaurants,Fast Food Resta...",2513 Highway 6 And 50,Grand Junction,81505,Colorado,US,39.081350,-108.586890,30,3.5,6,False
2,Chipotle Mexican Grill,"Fast Food Restaurants,Mexican Restaurants,Rest...",3455 N Salida Court,Aurora,80011,Colorado,US,39.763690,-104.776710,142,3,43,False
3,Taco Bell,"Fast Food Restaurants,Mexican Restaurants,Rest...",5225 E Colfax Avenue,Denver,80220,Colorado,US,39.740440,-104.926360,529,2,40,False
4,McDonald's,"Fast Food Restaurants,Hamburgers and Hot Dogs,...",1350 W Colfax Avenue,Denver,80204,Colorado,US,39.739670,-105.004670,784,2,56,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3994,Taco Bell,"Carry-out food,International restaurants,Mexic...",3645 N Atlantic Ave,Cocoa Beach,32931,Florida,US,28.352106,-80.608362,529,1.5,31,False
3995,Carl's Jr.,"Fast Food Restaurant,American Restaurant",639 N Pacific Coast Hwy,El Segundo,90245,California,US,33.924851,-118.396732,205,3,63,False
3996,White Castle,"Burger Joint,Fast Food Restaurant",43-02 Queens Blvd,Sunnyside,11104,New York,US,40.743144,-73.921320,28,3.5,105,False
3997,KFC,"Fried Chicken Joint,Take Out Restaurants,Fast ...",4835 NE Sandy Blvd,Portland,97213,Oregon,US,45.539722,-122.612437,376,2,23,False


In [7]:
#adding data from Yelp API into the second dataframe
params = {}
counter = 0
for index, row in renamed_fastfood_data_2.iterrows():
    restaurant_name = row['name']
    restaurant_lat = row['latitude']
    restaurant_lng = row['longitude']
    params['term'] = restaurant_name
    params['latitude'] = restaurant_lat
    params['longitude'] = restaurant_lng
    try:
        yelp_response = requests.get(yelp_url, headers = headers, params = params, timeout = 5)
        yelp_json = yelp_response.json()
        renamed_fastfood_data_2.loc[index, 'Rating'] = yelp_json['businesses'][0]['rating']
        renamed_fastfood_data_2.loc[index, 'Review Count'] = yelp_json['businesses'][0]['review_count']
        renamed_fastfood_data_2.loc[index, 'Closed'] = yelp_json['businesses'][0]['is_closed']
    except (KeyError, IndexError):
        renamed_fastfood_data_2.loc[index, 'Closed'] = True
        continue
renamed_fastfood_data_2.to_csv('Resources/fastfood_yelp_2.csv')
renamed_fastfood_data_2

,name,categories,address,city,postalCode,state,country,latitude,longitude,count,Rating,Review Count,Closed
0,Panera Bread,"Breakfast,Cafe,Bakery,Caf,Take Out Restaurants...",3942 Townsfair Way,Columbus,43219,Ohio,US,40.049258,-82.916792,140,2.5,42,False
1,Arby's,"Delis And Delicatessens,Restaurant,Fast Food R...",603 S George Nigh Expy,McAlester,74501,Oklahoma,US,34.916276,-95.746002,252,3.5,7,False
2,Pizza Hut,"Carry-out food,Caterers,Restaurant delivery se...",1831 Beall Ave,Wooster,44691,Ohio,US,40.821870,-81.933747,474,2.5,13,False
3,Pizza Hut,"Pizza,Carry-out food,Restaurant,Caterers,Resta...",14923 Pacific Ave S,Tacoma,98444,Washington,US,47.121221,-122.434370,474,2,16,False
4,Pizza Hut,"Carry-out food,Caterers,Restaurant delivery se...",786 S Church St,Forest City,28043,North Carolina,US,35.319737,-81.880982,474,2.5,7,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4241,Subway,"Carry-out food,Delicatessens,Take Out Restaura...",202 W Camp Wisdom Rd,Duncanville,75116,Texas,US,32.661820,-96.908570,1219,1,3,False
4242,Bob Evans,"American Restaurants,Carry-out food,Restaurant...",121 Marlene Dr,Nicholasville,40356,Kentucky,US,37.954950,-84.537840,40,2.5,24,False
4243,McDonald's,"Carry-out food,Hot dog restaurants,Fast Food R...",2601 Plover Rd,Plover,54467,Wisconsin,US,44.453180,-89.543170,784,1.5,3,False
4244,Pizza Hut,"Carry-out food,Caterers,Restaurant delivery se...",170 E Main St,Hendersonville,37075,Tennessee,US,36.308262,-86.607339,474,2.5,13,False
